In [2]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [3]:
DATA_PATH = '../../../data/'
df_train = pd.read_csv(DATA_PATH + 'news_clf/news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv(DATA_PATH + 'news_clf/news_clustering_test.tsv', sep='\t')

df_train.info(), print('\n'), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   1800 non-null   int64 
 1   class   1800 non-null   object
 2   title   1800 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.3+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   600 non-null    int64 
 1   class   600 non-null    object
 2   title   600 non-null    object
dtypes: int64(1), object(2)
memory usage: 14.2+ KB


(None, None, None)

In [4]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [6]:
ws = WS(DATA_PATH+'ckip_data/')
pos = POS(DATA_PATH+'ckip_data/')

In [7]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [8]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

---
# 尋找降維的詞向量：PPMI + SVD

In [9]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [10]:
len(word2index)

6690

如果使用one-hot就需要這麼大的維度的詞向量

In [32]:
# 建立Co-Matrix

vocab_size = len(word2index)
co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

window_size = 1
# YOUR CODE HERE

for pairs in train_title_cuts.values():
    words, _ = list(zip(*pairs))
    for index, word in enumerate(words):
        prev_index = index - window_size if index - window_size >= 0 else 0
        prev_words = words[prev_index:index]
        word_id = word2index[word]
        for prev_word in prev_words:
            prev_id = word2index[prev_word]
            co_matrix[word_id, prev_id] += 1
            co_matrix[prev_id, word_id] += 1

# END YOUR CODE

In [33]:
co_matrix

array([[0, 2, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [34]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
    # YOUR CODE HERE
    
    M = np.zeros_like(co_matrix, dtype=np.float32)
    N = np.sum(co_matrix)
    S = np.sum(co_matrix, axis=0)
    total = co_matrix.shape[0] * co_matrix.shape[1]

    for i in tqdm(range(co_matrix.shape[0])):
        for j in range(co_matrix.shape[1]):
            pmi = np.log2(co_matrix[i, j] * N / (S[i] * S[j] + eps))
            M[i, j] = max(0, pmi)
    
    # END YOUR CODE
    return M

ppmi = get_ppmi(co_matrix)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log2
  del sys.path[0]


In [35]:
ppmi

array([[ 0.      ,  9.267157,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 9.267157,  0.      , 11.515084, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.515084,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [36]:
ppmi.shape

(6690, 6690)

In [37]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到 dim=1000
# YOUR CODE HERE

svd = TruncatedSVD(n_components=1000)
word_vectors = svd.fit_transform(ppmi)

# END YOUR CODE

In [38]:
word_vectors.shape

(6690, 1000)

# 新的詞向量 + Group mean vector: 測試

In [39]:
excluded_flags = [
    'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
    'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
    'Caa', 'Cab', 'Cba', 'Cbb',
    'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
    'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
    'SPCHANGECATEGORY', 'WHITESPACE'
]

In [40]:
train_svd_vectors = {}
for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    train_svd_vectors[index] = vector

In [41]:
test_svd_vectors = {}
for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [42]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_svd_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)

In [43]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [44]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [46]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ')
    print('-', counter)
    accuracy = counter[group]/sum(counter[x] for x in counter)
    print('- Accuracy:', round(accuracy, 2), '\n')

predict 體育 : 
- Counter({'體育': 61, '遊戲': 10, '旅遊': 9, '財經': 8, '科技': 5, '農業': 4})
- Accuracy: 0.63 

predict 財經 : 
- Counter({'財經': 62, '科技': 24, '農業': 16, '體育': 8, '遊戲': 8, '旅遊': 7})
- Accuracy: 0.5 

predict 科技 : 
- Counter({'科技': 52, '體育': 15, '財經': 14, '農業': 9, '遊戲': 9, '旅遊': 8})
- Accuracy: 0.49 

predict 旅遊 : 
- Counter({'旅遊': 58, '農業': 11, '科技': 5, '財經': 4, '體育': 2, '遊戲': 2})
- Accuracy: 0.71 

predict 農業 : 
- Counter({'農業': 58, '旅遊': 7, '體育': 4, '財經': 4, '遊戲': 4, '科技': 2})
- Accuracy: 0.73 

predict 遊戲 : 
- Counter({'遊戲': 67, '科技': 11, '旅遊': 9, '體育': 8, '財經': 8, '農業': 1})
- Accuracy: 0.64 

